In [1]:
from random import shuffle
import pandas as pd
import plotly.express as px
import numpy as np
from tqdm import tqdm_notebook

pd.options.plotting.backend = "plotly"

In [31]:
class Hand(list):
    
    def popn(self, n):
        return [self.pop() for _ in range(n)]
    
    def add_cards(self, lst):
        for el in lst:
            self.insert(el, 0)
        

def deal():
    cards = list(range(1, 14)) * 4
    shuffle(cards)
    left, right = Hand(cards[:26]), Hand(cards[26:])
    assert len(left), len(right) == (26, 26)
    return left, right


In [32]:
def simuler():
    left_hand, right_hand = deal()
    outcome = [len(left_hand)]
    while len(left_hand) > 0 and len(right_hand) > 0:
        pot = []
        left, right = left_hand.pop(), right_hand.pop()
        pot += [left, right]
        while left == right:
            try:
                pot += left_hand.popn(3)
            except IndexError:
                outcome.append(0)
                return outcome
            try:
                pot += right_hand.popn(3)
            except IndexError:
                outcome.append(52)
                return outcome
            try:
                left = left_hand.pop()
            except IndexError:
                outcome.append(0)
                return outcome
            try:
                right = right_hand.pop()
            except IndexError:
                outcome.append(52)
                return outcome
            pot += [left, right]
        if left > right:
            left_hand.add_cards(pot)
        elif left < right:
            right_hand.add_cards(pot)
        outcome.append(len(left_hand))
    return outcome
            


In [44]:
N = 100_000
data = [simuler() for idx in tqdm_notebook(range(N))]
sims = pd.DataFrame(data).T.fillna(method="ffill").astype(int)
sims = pd.concat([sims])
sims.info()

/var/folders/lt/z_62r1wn3gn2lwvcrcmw7krc0000gn/T/ipykernel_86978/2747621178.py:2: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/100000 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Columns: 100000 entries, 0 to 99999
dtypes: int64(100000)
memory usage: 27.5 MB


In [45]:
sims.head()

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
0,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
1,25,27,25,25,25,27,25,27,27,27,...,27,27,27,27,27,25,27,25,25,25
2,26,26,24,26,26,26,26,26,32,26,...,26,26,28,28,26,24,26,24,24,20
3,25,27,25,25,27,27,27,27,27,25,...,27,25,29,27,25,25,25,23,25,21
4,26,28,24,24,28,26,26,26,26,24,...,32,30,30,28,26,24,24,22,26,20


In [46]:
sims.tail()

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
31,52,52,0,52,52,0,0,52,0,0,...,52,52,0,0,52,0,0,0,52,0
32,52,52,0,52,52,0,0,52,0,0,...,52,52,0,0,52,0,0,0,52,0
33,52,52,0,52,52,0,0,52,0,0,...,52,52,0,0,52,0,0,0,52,0
34,52,52,0,52,52,0,0,52,0,0,...,52,52,0,0,52,0,0,0,52,0
35,52,52,0,52,52,0,0,52,0,0,...,52,52,0,0,52,0,0,0,52,0


In [88]:
# Changes from hand to hand
sims.diff(1).stack().value_counts().sort_index().plot()

Converting all right winners to get everything on same scale

In [47]:
left_winner_cols = (sims.iloc[-1, :] == 52)
left_winners = sims.loc[:, left_winner_cols]
right_winners = sims.loc[:, ~left_winner_cols]
left_winner_cols.mean()

0.46987

In [49]:
sims_sym = pd.concat([left_winners, 52-right_winners], axis=1)
sims_sym.tail()

,0,1,3,4,7,14,16,17,18,25,...,99982,99983,99987,99989,99992,99993,99995,99996,99997,99999
31,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
32,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
33,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
34,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
35,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52


In [50]:
mins = sims_sym.min(axis=0)
mins.hist(title="The lowest number of cards the winner ever had")

In [51]:
winners = np.where(left_winner_cols, "left", "right")
sims_winner = sims.copy()
sims_winner.columns = winners
sims_winner.head()

,left,left,right,left,left,right,right,left,right,right,...,left,left,right,right,left,right,right,right,left,right
0,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
1,25,27,25,25,25,27,25,27,27,27,...,27,27,27,27,27,25,27,25,25,25
2,26,26,24,26,26,26,26,26,32,26,...,26,26,28,28,26,24,26,24,24,20
3,25,27,25,25,27,27,27,27,27,25,...,27,25,29,27,25,25,25,23,25,21
4,26,28,24,24,28,26,26,26,26,24,...,32,30,30,28,26,24,24,22,26,20


In [85]:
bayes = (
    sims_winner
    .T
    .stack()
    .reset_index()
    .groupby("level_0")[0].value_counts()
    .unstack(0)
    .T
    .fillna(0)
    .reset_index()
    .rename(columns={0: "left_cards"})
    .assign(
        prior_left=1,
        prior_right=1,
        post_left = lambda df: df.prior_left + df.left,
        post_right = lambda df: df.prior_right + df.right,
        prob_left = lambda df: df.post_left / (df.post_left + df.post_right),
        prob_right = lambda df: df.post_right / (df.post_left + df.post_right)
    )
)
bayes

level_0,left_cards,left,right,prior_left,prior_right,post_left,post_right,prob_left,prob_right
0,0,0.0,752499.0,1,1,1.0,752500.0,0.000001,0.999999
1,1,0.0,826.0,1,1,1.0,827.0,0.001208,0.998792
2,2,0.0,1057.0,1,1,1.0,1058.0,0.000944,0.999056
3,3,0.0,1383.0,1,1,1.0,1384.0,0.000722,0.999278
4,4,0.0,1781.0,1,1,1.0,1782.0,0.000561,0.999439
5,5,0.0,2176.0,1,1,1.0,2177.0,0.000459,0.999541
6,6,0.0,2595.0,1,1,1.0,2596.0,0.000385,0.999615
7,7,0.0,3031.0,1,1,1.0,3032.0,0.000330,0.999670
8,8,0.0,3712.0,1,1,1.0,3713.0,0.000269,0.999731
9,9,0.0,4483.0,1,1,1.0,4484.0,0.000223,0.999777


In [86]:
px.line(bayes, x="left_cards", y="prob_left", title="Probability of winning as function of cards in hand")